In [ ]:
import numpy as np 
import numpy.linalg as nl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as so
import scipy.linalg as sl
#import jax.numpy as jnp
#from jax import grad

import math
import sklearn.datasets

# Question 1

In [ ]:
# Gram-Schmidt Algorithm [IALA-PC]:

def gram_schmidt(a):
  q = []
  for i in range(len(a)):
    # Orthogonalization:
    q_tilde = a[i]
    for j in range(len(q)):
      q_tilde = q_tilde - (q[j] @ a[i])*q[j]
    # Test for Dependennce:
    if np.sqrt(sum(q_tilde**2)) <= 1e-10:
      #print("Vectors are linearly dependent.")
      #print("GS algorithm terminates at iteration ", i+1)
      return q
    # Normalization:
    else:
      q_tilde = q_tilde / np.sqrt(sum(q_tilde**2))
      q.append(q_tilde)
  #print("Vectors are linearly independent.")
  return q

In [ ]:
# QR Factorization Function [IALA-PC]:

def QR_factorization(A):
  Q_transpose = np.array(gram_schmidt(A.T))
  R = Q_transpose @ A
  Q = Q_transpose.T
  return Q, R

## Part (a) & Part (b):

**Resource:** Matrix Computations by Gene H. Golub and Charles F. Van Loan. 

Typically, the subspaces $F$ and $G$ are matrix ranges, e.g.,
$$
\begin{array}{ll}
F=\operatorname{ran}(A), & A \in \mathbb{R}^{k \times n} \\
G=\operatorname{ran}(B), & B \in \mathbb{R}^{k \times m} .
\end{array}
$$
The principal vectors and angles can be computed using the QR factorization and the SVD. Let $A=Q_{A} R_{A}$ and $B=Q_{B} R_{B}$ be thin QR factorizations and assume that
$$
Q_{A}^{T} Q_{B}=Y \Sigma Z^{T}=\sum_{i=1}^{q} \sigma_{i} y_{i} z_{i}^{T}
$$
is the SVD of $Q_{A}^{T} Q_{B} \in \mathbb{R}^{n \times m}$. Since $\left\|Q_{A}^{T} Q_{B}\right\|_{2} \leq 1$, all the singular values are between 0 and 1 and we may write $\sigma_{i}=\cos \left(\theta_{i}\right), i=1: m .$ Let
$$
\begin{aligned}
Q_{A} Y &\left.=\left|f_{1}\right| \cdots \mid f_{n}\right] \\
Q_{B} Z &=\left[g_{1}|\cdots| g_{m}\right]
\end{aligned}
$$

In light of the following theorem, Algorithm $6.4 .3$ can also be used to compute an orthonormal basis for $\operatorname{ran}(A) \cap \operatorname{ran}(B)$ where $A \in \mathbb{R}^{m \times p}$ and $B \in \mathbb{R}^{m \times q}$

**Theorem 6.4.2.** Let $\left\{\cos \left(\theta_{i}\right)\right\}_{i=1}^{q}$ and $\left\{f_{i}, g_{i}\right\}_{i=1}^{q}$ be defined by Algorithm 6.4.3. If the index $s$ is defined by $1=\cos \left(\theta_{1}\right)=\cdots=\cos \left(\theta_{s}\right)>\cos \left(\theta_{s+1}\right)$, then
$$
\operatorname{ran}(A) \cap \operatorname{ran}(B)=\operatorname{span}\left\{f_{1}, \ldots, f_{s}\right\}=\operatorname{span}\left\{g_{1}, \ldots, g_{s}\right\}
$$

In [ ]:
def intersect(A, B):
    QA, RA = QR_factorization(A)
    QB, RB = QR_factorization(B)
    C = QA.T@QB # Dimension n * m
    
    Y, s, Z_T = nl.svd(QA.T@QB) # Singular Values between 0 and 1 
    
    f = QA@Y # Orthonormal Columns (Principal Vectors)
    g = QB@Z_T.T # Orthonormal Columns (Principal Vectors)

    # Principal Angles:
    #thetas = sl.subspace_angles(A, B) 
    #print(thetas, "\n")
    
    # Orthonormal basis for ran(A) ∩ ran(B):
    #print("v_f:")
    v_f = []
    for i in range(0, f.shape[1]):
      v_f.append(f[:, i])
    #print(np.round(v_f, 3), "\n")

    #print("v_g:")
    v_g = []
    for i in range(0, g.shape[1]):
      v_g.append(g[:, i])
    #print(np.round(v_g,3), "\n")
    
    #Span:
    print("Orthonormal basis for ran(A) ∩ ran(B):")
    span = []
    for i in range(np.shape(v_f)[0]):
        for j in range(np.shape(v_g)[0]):
            if np.allclose(v_f[i],v_g[j]) == True:
                span.append(v_f[i].reshape(-1, 1))
    
    if span != []:
        print("Shape:", np.hstack(span).shape)
        U = np.hstack(span)
    else: U = []
    return U


In [ ]:
# Note: v is taken here as column vector.
def verify(v, U):
    aug = np.concatenate((U, v), axis=1)
    return np.linalg.matrix_rank(U) != np.linalg.matrix_rank(U[:,:-1])

## Part (c):

In [ ]:
A1 = np.load("Q1_A1.npy")
print("Shape of A1:", A1.shape)
print(A1)

Shape of A1: (5, 4)
[[ 0.72691914  0.76570693  1.66844929 -1.89172172]
 [ 0.49642416 -0.25253499  0.26595154  0.86727004]
 [ 0.56951769 -1.38907031  0.51185823  3.73950234]
 [-1.58890387  1.74739157  1.33673492 -1.71442837]
 [ 2.3303401   1.72619661 -0.32390699  1.49149046]]


In [ ]:
B1 = np.load("Q1_B1.npy")
print("Shape of B1:", B1.shape)
print(B1)

Shape of B1: (5, 3)
[[ 3.67389478  1.1382018   0.84157615]
 [ 2.10899494 -0.70562447 -2.09429783]
 [ 2.56734678 -0.70711484 -2.29453434]
 [ 4.17015366  2.89770021 -2.66491727]
 [-2.16499894  0.29052221 -4.15697366]]


In [ ]:
A2 = np.load("Q1_A2.npy")
print("Shape of A2:", A2.shape)
print(A2)

Shape of A2: (6, 2)
[[-1. -1.]
 [ 1.  1.]
 [-1. -1.]
 [-2.  0.]
 [ 4.  1.]
 [ 2. -4.]]


In [ ]:
B2 = np.load("Q1_B2.npy")
print("Shape of B2:", B2.shape)
print(B2)

Shape of B2: (6, 3)
[[-0.2894602   0.18336193 -0.31367473]
 [-0.22877838 -0.28691597  0.83167912]
 [ 0.90195025  0.00565713  0.03488205]
 [-0.0439997   0.90152785  0.26818022]
 [ 0.15902412  0.24335181  0.31210842]
 [-0.1520998   0.10978641 -0.19841612]]


### Verify Test Case 1:


In [ ]:
U1 = intersect(A1, B1)
print(U1)

Orthonormal basis for ran(A) ∩ ran(B):
Shape: (5, 2)
[[ 0.17554948  0.58741637]
 [ 0.03035738  0.03457349]
 [ 0.04044495  0.07428885]
 [-0.59947778  0.6987357 ]
 [-0.77926338 -0.39999534]]


In [ ]:
v1 = U1[:, 0].reshape(-1, 1)
print(v1)

[[ 0.17554948]
 [ 0.03035738]
 [ 0.04044495]
 [-0.59947778]
 [-0.77926338]]


In [ ]:
v2 = U1[:, 1].reshape(-1, 1)
print(v2)

[[ 0.58741637]
 [ 0.03457349]
 [ 0.07428885]
 [ 0.6987357 ]
 [-0.39999534]]


In [ ]:
verify(v1, U1)

True

In [ ]:
verify(v2, U1)

True

### Verify Test Case 2:


In [ ]:
U2 = intersect(A2, B2)
print(U2)

Orthonormal basis for ran(A) ∩ ran(B):
[]
